<a href="https://colab.research.google.com/github/bbeck0744/Lake_Minnetonka_Model/blob/main/Mass_Balance_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Retrieving Recent Weather Data and Forecasted Weather Data**
---

In [17]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime, timedelta, tzinfo
import joblib
import pytz
import pandas as pd
from pandas import read_csv, DataFrame, concat
import json
import requests
import time
from urllib.request import urlopen
from functools import partial, reduce
import requests
from pandas.plotting import register_matplotlib_converters
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from numpy import loadtxt
import os, sys, io, csv

In [18]:
#create dataframe that has weather station locations
df1 = pd.DataFrame({
        "site": ["msp", "chan"],
        "is_site": ["MSP", "FCM"],
        "lat": ["-93.23", "-93.57"],
        "long": ["44.8914", "44.8491"],
        "x": ["53", "97"],
        "y": ["77", "65"]
    })

# Print the Timestamp object
now = datetime.utcnow().replace(microsecond=0, second=0, minute=0)

#obtain data from the past 14 days
start = now - timedelta(days=14)
#create a datetime in the future to ensure that timestamp gets the most up-to-date weather measurement
now = now + timedelta(days=1)
now_string = now.strftime("%d/%m/%Y %H:%M")
start_string = start.strftime("%d/%m/%Y %H:%M")


base_url = "https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py"
site_data = "data=tmpc&data=relh&data=sped&data=p01m"
end_url = "tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=null&trace=null&direct=no&report_type=1&report_type=2"

#create empty dataframe

df_weather = pd.DataFrame(columns = ["valid"])

for site in range(0, len(df1)):
    try:
        station = df1.at[site, 'is_site']
        #build url
        build_url = f"{base_url}?station={station}&{site_data}&year1={start.year}&month1={start.month}&day1={start.day}&year2={now.year}&month2={now.month}&day2={now.day}&{end_url}"
        #request url
        print(build_url)
        data_request = urlopen(build_url, timeout=300).read().decode("utf-8")
        #parse data request csv string
        data = io.StringIO(data_request)
        #read csv
        df = pd.read_csv(data, sep=",")
        #drop any dates with blank temps
        df=df.dropna(subset=['tmpc'])
        #convert time column to datetime
        df['valid'] = pd.to_datetime(df['valid'], utc=True)
        #only take hourly data
        df = df[df['valid'].dt.minute >= 53]
        #fill windspeed datetimes to zero
        df["sped"].fillna("0", inplace = True)
        #fill precip nulls to zero
        df["p01m"].fillna("0", inplace = True)
        df['sped'] = pd.to_numeric(df['sped'],errors = 'coerce')
        df['sped'] = 1.60934 * df['sped']
        df['p01m'] = pd.to_numeric(df['p01m'],errors = 'coerce')
        #round datetimes to the nearest hour
        df['valid']=df.valid.dt.round('1h')
        f = {'tmpc': 'mean', 'relh': 'mean', 'sped': 'mean', 'p01m': 'sum'}
        df.groupby(['valid'], as_index=False).agg(f)
        df = df.drop(columns=['station'])
        #append data to df_weather dataframe
        df_weather = df_weather.merge(df, how='outer', on='valid', suffixes=(None,f"_{station}"))
    except:
        print(f"{station} didn't work :(")
    continue
df_weather=df_weather.fillna(method='ffill')
#df = df.drop(columns=['station'])
df_precip = df_weather.drop(columns=['tmpc','relh','sped','tmpc_FCM','relh_FCM','sped_FCM','p01m_FCM'])
df_precip['valid'] = pd.to_datetime(df_precip['valid'])
df_precip['valid'] = df_precip['valid'].dt.round('H')


https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=MSP&data=tmpc&data=relh&data=sped&data=p01m&year1=2024&month1=5&day1=20&year2=2024&month2=6&day2=4&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=null&trace=null&direct=no&report_type=1&report_type=2
https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=FCM&data=tmpc&data=relh&data=sped&data=p01m&year1=2024&month1=5&day1=20&year2=2024&month2=6&day2=4&tz=Etc%2FUTC&format=onlycomma&latlon=no&elev=no&missing=null&trace=null&direct=no&report_type=1&report_type=2


In [19]:
import pandas as pd
from datetime import datetime, timedelta

# Calculate the date range
today = datetime.now()
start_date = today - timedelta(days=7)
end_date = today + timedelta(days=8)

# Format the dates
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

# API URL with dynamic date range
url = f"https://gis.minnehahacreek.org/KiWIS/KiWIS?service=kisters&type=queryServices&request=getTimeseriesValues&datasource=0&format=json&ts_id=6610010&from={start_date_str}&to={end_date_str}&metadata=false"

# Fetch the data from the API
response = requests.get(url)
data = response.json()

# Extract the relevant information
data_points = data[0]['data']

# Create lists to hold the timestamps and values
timestamps = []
values = []

# Extract the timestamps and values
for data_point in data_points:
    timestamps.append(data_point[0])
    values.append(data_point[1])

# Combine the data into a DataFrame
evap_df = pd.DataFrame({
    'Timestamp': timestamps,
    'evap': values
})

# Display the DataFrame
evap_df['Timestamp'] = pd.to_datetime(evap_df['Timestamp'])
evap_df['Timestamp'] = evap_df['Timestamp'].dt.tz_convert('UTC')
evap_df['Timestamp']=evap_df.Timestamp.dt.floor('D')
evap_df['Timestamp']=evap_df['Timestamp'].dt.round('D')
print(url)


https://gis.minnehahacreek.org/KiWIS/KiWIS?service=kisters&type=queryServices&request=getTimeseriesValues&datasource=0&format=json&ts_id=6610010&from=2024-05-27&to=2024-06-11&metadata=false


In [20]:
# List of ts_id values
ts_ids = ['6599010', '6601010', '6603010', '6607010', '6605010', '6595010', '6597010']

# Calculate the date range
today = datetime.now()
start_date = today - timedelta(days=7)
end_date = today + timedelta(days=8)

# Format the dates
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date_str = end_date.strftime('%Y-%m-%d')

# Initialize an empty dictionary to hold all data
all_data = {}

# Iterate through the list of ts_id values
for ts_id in ts_ids:
    # API URL with dynamic date range
    url = f"https://gis.minnehahacreek.org/KiWIS/KiWIS?service=kisters&type=queryServices&request=getTimeseriesValues&datasource=0&format=json&ts_id={ts_id}&from={start_date_str}&to={end_date_str}&metadata=false"

    # Fetch the data from the API
    response = requests.get(url)
    data = response.json()

    # Extract the relevant information
    data_points = data[0]['data']

    # Create lists to hold the timestamps and values
    timestamps = []
    values = []

    # Extract the timestamps and values
    for data_point in data_points:
        timestamps.append(data_point[0])
        values.append(data_point[1])

    # Combine the data into a DataFrame for the current ts_id
    df = pd.DataFrame({
        'Timestamp': timestamps,
        ts_id: values
    })
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    # Merge with the all_data dictionary
    if 'Timestamp' not in all_data:
        all_data['Timestamp'] = df['Timestamp']

    all_data[ts_id] = df[ts_id]

# Convert the dictionary to a DataFrame
combined_df = pd.DataFrame(all_data)



In [21]:
#MCWD ArcGIS Portal Data
dam_url = "https://gis.minnehahacreek.org/server/rest/services/Hosted/Grays_Bay_Dam_Operations_Public/FeatureServer/query?layerDefs=%7B%220%22%3A%22q_cfs%3E%3D0%22%7D&geometryType=esriGeometryEnvelope&spatialRel=esriSpatialRelIntersects&returnDistinctValues=false&returnGeometry=true&returnIdsOnly=false&returnCountOnly=false&returnZ=false&returnM=false&multipatchOption=xyFootprint&returnTrueCurves=false&sqlFormat=none&f=pjson"


# GET request
dam_response = requests.get(dam_url).json()

flow = []
date_time = []

#parsing json and creating lists
for num_feature in range(0, len(dam_response["layers"][0]["features"])):
  date_time.append(pd.to_datetime(dam_response["layers"][0]["features"][num_feature]["attributes"]["r_datetime"] / 1000, unit='s', utc=True))
  flow.append(dam_response["layers"][0]["features"][num_feature]["attributes"]["q_cfs"])



dam_dict = {
    "date": date_time,
    "flow": flow
}

#creating a dataframe from the the lists and dictionaries
site_df = pd.DataFrame(dam_dict)
site_df['date']=site_df.date.dt.floor('1h')
site_df['date'] = pd.to_datetime(site_df['date'])
site_df['date'] = site_df['date'].dt.round('H')


In [22]:
#combining dataframes
df_past = df_precip.merge(site_df, how='left', left_on='valid', right_on='date')
df_past.drop(["date"], axis=1, inplace=True)



In [23]:
# Fill NaN values in other columns using forward fill
df_past = df_past.fillna(method='ffill')

# Fill NaN values in the "flow" column with zeros
df_past = df_past.fillna(method='bfill')


In [24]:
# set parameters
ndays = 14
lake_id = "05289000"
elevation_id = "00065"

now = datetime.utcnow().replace(microsecond=0, second=0, minute=0)

#obtain data from the past 14 days
start = now - timedelta(days=ndays)
#create a datetime in the future to ensure that timestamp gets the most up-to-date weather measurement
now_string = now.strftime("%Y-%m-%d")
start_string = start.strftime("%Y-%m-%d")

creek_url = f"https://waterservices.usgs.gov/nwis/iv/?sites={lake_id}&parameterCd={elevation_id}&format=json&startDT={start_string}&endDT={now_string}"

# GET request
creek_response = requests.get(creek_url).json()

elevation = []
date_time = []

#parsing json and creating lists
for num_feature in range(0, len(creek_response['value']['timeSeries'][0]['values'][0]['value'])):
  date_time.append(creek_response['value']['timeSeries'][0]['values'][0]['value'][num_feature]['dateTime'])
  elevation.append(creek_response['value']['timeSeries'][0]['values'][0]['value'][num_feature]['value'])


elevation = [None if el == '' else float(el) for el in elevation]
data_tuples = list(zip(date_time,elevation))
df_mtk = pd.DataFrame(data_tuples, columns=['date','elevation'])
df_mtk['date'] = pd.to_datetime(df_mtk['date'], utc=True)
df_mtk = df_mtk[df_mtk['elevation'] != -999999.0]
df_mtk = df_mtk.groupby(df_mtk.date.dt.floor("h")).mean()
#df_mtk
df_mtk=df_mtk.reset_index(drop=True)

df_mtk.columns = ['rounded_time', 'elevation']


In [25]:
df_past['flow'] = df_past['flow'] * 60 * 60 / 43559.9
df_past['p01m'] = df_past['p01m'] * 0.0393701


In [26]:
df_mtk['rounded_time'] = df_mtk['rounded_time'].dt.round('D')
df_past['valid'] = df_past['valid'].dt.round('D')
df_past_grouped = df_past.groupby('valid').sum()
df_mtk_grouped = df_mtk.groupby('rounded_time').mean()
df_mtk_grouped=df_mtk_grouped.reset_index(drop=False)
df_past_grouped=df_past_grouped.reset_index(drop=False)


In [27]:
df_past = df_past_grouped.merge(df_mtk_grouped, how='inner', left_on='valid', right_on='rounded_time')

df_past.drop(["rounded_time"], axis=1, inplace=True)
df_past = df_past.sort_values(by="valid")
#df_past.columns = ['DATE', 'TAVG', 'relh', 'sped', 'PRCP', 'TAVG_c', 'relh_c', 'sped_c', 'PRCP_c', 'Dam_Discharge','elevation']



In [28]:
# Createthe Timestamp object
ts = pd.Timestamp(datetime.utcnow())

# Print the Timestamp object
forecast_start = ts.floor(freq = 'H')
forecast_end = forecast_start + timedelta(days=7)

input_df = pd.DataFrame(pd.date_range(forecast_start, forecast_end, freq='1H'), columns=['date_time'])
input_df['date_time'] = pd.to_datetime(input_df['date_time'], utc=True)


date_time_prediction = []
value_prediction = []
#properties = []

d={}


parameters = ['temperature', 'quantitativePrecipitation', 'relativeHumidity', 'windSpeed']

base_url = "https://api.weather.gov/gridpoints/MPX/"

forecast_coord_weather_url = "https://api.weather.gov/gridpoints/MPX/"
prediction_num = 0
for site in range(0, len(df1)):
    properties = []
    x = df1.at[site, 'x']
    y = df1.at[site, 'y']
    site = df1.at[site, 'site']

    forecast_coord_weather_url = f"{base_url}{x},{y}"
    print(forecast_coord_weather_url)
    forecast_json_data = requests.get(forecast_coord_weather_url).json()
    print(forecast_json_data)
    df = {}
    input_df = pd.DataFrame(pd.date_range(forecast_start, forecast_end, freq='1H'), columns=['date_time'])
    input_df['date_time'] = pd.to_datetime(input_df['date_time'], utc=True)
    for parameter in parameters:
        #properties = []
        value_prediction = []
        date_time_prediction = []
        for num_forecasts in range(0, len(forecast_json_data['properties'][parameter]['values'])):


            date_time_prediction.append(forecast_json_data['properties'][parameter]['values'][num_forecasts]['validTime'])

            #prediction data
            value_prediction.append(forecast_json_data['properties'][parameter]['values'][num_forecasts]['value'])
        dict = {'date_time': date_time_prediction, f'{parameter}_{site}': value_prediction}
        df[parameter] = pd.DataFrame(dict)
        split_string = df[parameter]['date_time'].str.split('/', n=1, expand=True)
        df[parameter]['date_time'] = split_string[0]
        df[parameter][f'period_length_{site}_{parameter}'] = pd.to_numeric(split_string[1].str[2], errors='coerce')
        df[parameter][f'date_time'] = pd.to_datetime(df[parameter]['date_time'], utc = True)

        input_df = input_df.merge(df[parameter], how='outer', left_on='date_time', right_on='date_time')
        prediction_num += 1
    d[site] = pd.DataFrame(input_df)

df_predictions = pd.DataFrame()
my_reduce = partial(pd.merge, on='date_time', how='inner')
df_predictions = reduce(my_reduce, d.values())

#take predictions and create a processed dataframe
df_predictions['chan_precip'] = df_predictions['quantitativePrecipitation_chan'] / df_predictions['period_length_chan_quantitativePrecipitation']
df_predictions['msp_precip'] = df_predictions['quantitativePrecipitation_msp'] / df_predictions['period_length_msp_quantitativePrecipitation']
df_predictions = df_predictions.drop(columns=['period_length_chan_temperature', 'period_length_msp_temperature',
                      'period_length_chan_quantitativePrecipitation', 'period_length_msp_quantitativePrecipitation',
                      'period_length_msp_windSpeed', 'period_length_chan_windSpeed', 'period_length_msp_relativeHumidity',
                                              'period_length_chan_relativeHumidity'])
df_predictions['chan_precip'] = df_predictions['chan_precip'].fillna(method='ffill')
df_predictions['msp_precip'] = df_predictions['msp_precip'].fillna(method='ffill')
df_predictions = df_predictions.drop(columns=['quantitativePrecipitation_msp', 'quantitativePrecipitation_chan'])

#reformat
df_predictions = df_predictions.interpolate(method ='ffill', limit_direction ='forward', axis=0)
df_predictions.columns = ['DATE','TAVG', 'relh', 'sped', 'TAVG_c', 'relh_c', 'sped_c', 'PRCP_c', 'PRCP']
df_predictions = df_predictions.drop(columns=['TAVG', 'relh', 'sped', 'TAVG_c', 'relh_c', 'sped_c', 'PRCP_c'])
df_predictions=df_predictions.fillna(method='bfill')


https://api.weather.gov/gridpoints/MPX/53,77
{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld', {'@version': '1.1', 'wmoUnit': 'https://codes.wmo.int/common/unit/', 'nwsUnit': 'https://api.weather.gov/ontology/unit/'}], 'id': 'https://api.weather.gov/gridpoints/MPX/53,77', 'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-94.9549201, 45.1013661], [-94.95493019999999, 45.080002], [-94.92468199999999, 45.0799925], [-94.92466519999999, 45.101356700000004], [-94.9549201, 45.1013661]]]}, 'properties': {'@id': 'https://api.weather.gov/gridpoints/MPX/53,77', '@type': 'wx:Gridpoint', 'updateTime': '2024-06-03T13:28:35+00:00', 'validTimes': '2024-06-03T07:00:00+00:00/P7DT18H', 'elevation': {'unitCode': 'wmoUnit:m', 'value': 367.8936}, 'forecastOffice': 'https://api.weather.gov/offices/MPX', 'gridId': 'MPX', 'gridX': '53', 'gridY': '77', 'temperature': {'uom': 'wmoUnit:degC', 'values': [{'validTime': '2024-06-03T07:00:00+00:00/PT1H', 'value': 18.3333333333

In [29]:
now = datetime.utcnow().replace(microsecond=0, second=0, minute=0)
rng = pd.date_range(now, periods=7, freq='D')

In [30]:
dam_date = []
fmt = '%Y-%m-%d'
e = pytz.timezone('US/Central')

for x in range(0, 7):

  now = datetime.utcnow().replace(microsecond=0, second=0, minute=0)
  now.strftime(fmt)
  date = now + timedelta(days=x)
  dam_date.append(date.strftime(fmt))


## **Setting dam Discharge**

---

*Please enter the Grays Bay Dam discharge for the upcoming 7 days. Press the play button when finished.*

In [31]:
Day_one   = 70 #@param {type:"slider", min:0, max:300, step:10}
Day_two   = 70 #@param {type:"slider", min:0, max:300, step:10}
Day_three = 70 #@param {type:"slider", min:0, max:300, step:10}
Day_four  = 70 #@param {type:"slider", min:0, max:300, step:10}
Day_five  = 70 #@param {type:"slider", min:0, max:300, step:10}
Day_six   = 70 #@param {type:"slider", min:0, max:300, step:10}
Day_seven = 70 #@param {type:"slider", min:0, max:300, step:10}

## **Entering Dam Discharge Estimates into Models**

---

In [42]:
df_dam_forecast = pd.DataFrame({ 'Date': rng, 'Dam_Discharge': [Day_one, Day_two, Day_three, Day_four, Day_five, Day_six, Day_seven] })


In [43]:
df_dam_forecast['Date'] = pd.to_datetime(df_dam_forecast['Date'], utc = True)
df_prediction = df_predictions.merge(df_dam_forecast, how='left', left_on='DATE', right_on='Date')
df_prediction = df_prediction.sort_values(by="DATE")
#Dropping duplicative date field
df_prediction.drop(["Date"], axis=1, inplace=True)
#back filling and forward filling discharge flows since dam discharge is inserted as a discrete date and needs to be converted into a continuous
df_prediction=df_prediction.fillna(method='ffill')
df_prediction=df_prediction.fillna(method='bfill')
lake_minnetonka = 14373
lake_minnetonka_drainage_areas = 32515
upper_watershed_drainage = 46344
runoff_area = upper_watershed_drainage + lake_minnetonka_drainage_areas - lake_minnetonka
df_prediction['DD_sim'] = df_prediction['Dam_Discharge'] * 0.0826448
df_prediction['PRCP'] = df_prediction['PRCP'] * 0.0393701
df_prediction['DD_0'] = 0 * 0.0
df_prediction['DD_10']= 10 * 0.0826448
df_prediction['DD_20'] = 20 * 0.0826448
df_prediction['DD_50'] = 50 * 0.0826448
df_prediction['DD_100'] = 100 * 0.0826448
df_prediction['DD_150'] = 150 * 0.0826448
df_prediction['DD_200'] = 200 * 0.0826448
df_prediction['DD_250'] = 250 * 0.0826448
df_prediction['DD_300'] = 300 * 0.0826448

df_prediction = df_prediction.drop(columns=['Dam_Discharge'])

In [44]:
df_prediction['DATE'] = pd.to_datetime(df_prediction['DATE']).dt.date
df_prediction_grouped = df_prediction.groupby('DATE').sum().reset_index()
df_prediction_grouped['DATE'] = pd.to_datetime(df_prediction_grouped['DATE'], utc=True)

In [45]:
df_prediction_grouped['runoff_vol'] = ((df_prediction_grouped['PRCP'] / 12) * runoff_area * 0.14) + (lake_minnetonka * (df_prediction_grouped['PRCP'] / 12))
df_prediction_grouped['delay_1'] = df_prediction_grouped['runoff_vol'] / 3
df_prediction_grouped['delay_2'] =  df_prediction_grouped['delay_1'].shift(1)
df_prediction_grouped['delay_3'] =  df_prediction_grouped['delay_1'].shift(2)
df_prediction_grouped['delay_2'].fillna(0, inplace=True)
df_prediction_grouped['delay_3'].fillna(0, inplace=True)
df_prediction_grouped['runoff_sum'] = df_prediction_grouped['delay_1'] + df_prediction_grouped['delay_2'] + df_prediction_grouped['delay_3']
merged_prediction = df_prediction_grouped.merge(evap_df, how='left', left_on='DATE', right_on='Timestamp')
merged_prediction = merged_prediction.drop(columns=['Timestamp'])

last_value = df_past['elevation'].iloc[-1]
merged_prediction.at[0, 'starting_value'] = last_value
merged_prediction['starting_value'].fillna(0, inplace = True)

In [46]:
merged_prediction['Level_sim'] = (merged_prediction['runoff_sum'] - merged_prediction['DD_sim'] - merged_prediction['evap']) / lake_minnetonka
merged_prediction['Level_0'] = (merged_prediction['runoff_sum'] - merged_prediction['DD_0'] - merged_prediction['evap']) / lake_minnetonka
merged_prediction['Level_10'] = (merged_prediction['runoff_sum'] - merged_prediction['DD_10'] - merged_prediction['evap']) / lake_minnetonka
merged_prediction['Level_20'] = (merged_prediction['runoff_sum'] - merged_prediction['DD_20'] - merged_prediction['evap']) / lake_minnetonka
merged_prediction['Level_50'] = (merged_prediction['runoff_sum'] - merged_prediction['DD_50'] - merged_prediction['evap']) / lake_minnetonka
merged_prediction['Level_100'] = (merged_prediction['runoff_sum'] - merged_prediction['DD_100'] - merged_prediction['evap']) / lake_minnetonka
merged_prediction['Level_150'] = (merged_prediction['runoff_sum'] - merged_prediction['DD_150'] - merged_prediction['evap']) / lake_minnetonka
merged_prediction['Level_200'] = (merged_prediction['runoff_sum'] - merged_prediction['DD_200'] - merged_prediction['evap']) / lake_minnetonka
merged_prediction['Level_250'] = (merged_prediction['runoff_sum'] - merged_prediction['DD_250'] - merged_prediction['evap']) / lake_minnetonka
merged_prediction['Level_300'] = (merged_prediction['runoff_sum'] - merged_prediction['DD_300'] - merged_prediction['evap']) / lake_minnetonka


In [47]:
merged_prediction['Minnetonka_simulation'] = (merged_prediction['Level_sim'] + merged_prediction['starting_value']).cumsum()
merged_prediction['Minnetonka_0'] = (merged_prediction['Level_0'] + merged_prediction['starting_value']).cumsum()
merged_prediction['Minnetonka_10'] = (merged_prediction['Level_10'] + merged_prediction['starting_value']).cumsum()
merged_prediction['Minnetonka_20'] = (merged_prediction['Level_20'] + merged_prediction['starting_value']).cumsum()
merged_prediction['Minnetonka_50'] = (merged_prediction['Level_50'] + merged_prediction['starting_value']).cumsum()
merged_prediction['Minnetonka_100'] = (merged_prediction['Level_100'] + merged_prediction['starting_value']).cumsum()
merged_prediction['Minnetonka_150'] = (merged_prediction['Level_150'] + merged_prediction['starting_value']).cumsum()
merged_prediction['Minnetonka_200'] = (merged_prediction['Level_200'] + merged_prediction['starting_value']).cumsum()
merged_prediction['Minnetonka_250'] = (merged_prediction['Level_250'] + merged_prediction['starting_value']).cumsum()
merged_prediction['Minnetonka_300'] = (merged_prediction['Level_300'] + merged_prediction['starting_value']).cumsum()


In [48]:
# Create a figure for level predictions
lev_fig = go.Figure()
lev_fig.add_trace(go.Scatter(x=merged_prediction['DATE'], y=merged_prediction['Minnetonka_0'], mode='lines', name='GBD: 0 CFS', line={'color': 'gray'}, opacity=0.2, line_shape='spline'))
lev_fig.add_trace(go.Scatter(x=merged_prediction['DATE'], y=merged_prediction['Minnetonka_10'], mode='lines', name='GBD: 10 CFS', line={'color': 'gray'}, opacity=0.25, line_shape='spline'))
lev_fig.add_trace(go.Scatter(x=merged_prediction['DATE'], y=merged_prediction['Minnetonka_20'], mode='lines', name='GBD: 20 CFS', line={'color': 'gray'}, opacity=0.3, line_shape='spline'))
lev_fig.add_trace(go.Scatter(x=merged_prediction['DATE'], y=merged_prediction['Minnetonka_50'], mode='lines', name='GBD: 50 CFS', line={'color': 'gray'}, opacity=0.4, line_shape='spline'))
lev_fig.add_trace(go.Scatter(x=merged_prediction['DATE'], y=merged_prediction['Minnetonka_100'], mode='lines', name='GBD: 100 CFS', line={'color': 'gray'}, opacity=0.5, line_shape='spline'))
lev_fig.add_trace(go.Scatter(x=merged_prediction['DATE'], y=merged_prediction['Minnetonka_150'], mode='lines', name='GBD: 150 CFS', line={'color': 'gray'}, opacity=0.6, line_shape='spline'))
lev_fig.add_trace(go.Scatter(x=merged_prediction['DATE'], y=merged_prediction['Minnetonka_200'], mode='lines', name='GBD: 200 CFS', line={'color': 'gray'}, opacity=0.7, line_shape='spline'))
lev_fig.add_trace(go.Scatter(x=merged_prediction['DATE'], y=merged_prediction['Minnetonka_250'], mode='lines', name='GBD: 250 CFS', line={'color': 'gray'}, opacity=0.8, line_shape='spline'))
lev_fig.add_trace(go.Scatter(x=merged_prediction['DATE'], y=merged_prediction['Minnetonka_300'], mode='lines', name='GBD: 300 CFS', line={'color': 'gray'}, opacity=0.9, line_shape='spline'))
lev_fig.add_trace(go.Scatter(x=merged_prediction['DATE'],y=merged_prediction['Minnetonka_simulation'], mode='lines', name='User Simulation', line={'color': 'rgb(0,173,236)'}, line_shape='spline'))
lev_fig.update_layout(title="Lake Minnetonka Level Scenario Output",xaxis_title="Date",yaxis_title="Lake Elevation (ft)",barmode='stack', showlegend=True)

# Create a figure for tracking mass transfer
mass_fig = go.Figure()
mass_fig.add_trace(go.Bar(x=merged_prediction['DATE'], y=merged_prediction['runoff_sum'],name='Lake Inflow',marker_color='rgb(142,199,255)'))
mass_fig.add_trace(go.Bar(x=merged_prediction['DATE'], y=merged_prediction['evap'], name='Lake Evaporation',marker_color='rgb(250,161,38)', opacity=0.85))
mass_fig.add_trace(go.Bar(x=merged_prediction['DATE'], y=merged_prediction['DD_sim'], name='Dam Discharge',marker_color='rgb(250,161,38)',opacity=1))
mass_fig.update_layout(title="Lake Inflow, Evaporation, and Dam Discharge",xaxis_title="Date",yaxis_title="Runoff Sum",barmode='stack', showlegend=True)

## **Output Figures**
*Press play button below to create figures*
---


In [41]:
lev_fig.show()
mass_fig.show()